In [19]:
import pandas as pd

# Read the CSVs into a variable
price_data = pd.read_csv('hotels-europe_price.csv')
features_data = pd.read_csv('hotels-europe_features.csv')

In [20]:
# merge the two datasets with the key 'hotel_id'
merged_hotel_data = pd.merge(price_data, features_data, on='hotel_id')

In [21]:
# convert accommodation_type and city_actual to string so that it can be filtered
merged_hotel_data['accommodation_type'] = merged_hotel_data['accommodation_type'].astype('string')
merged_hotel_data['city_actual']=merged_hotel_data['city'].astype('string')

In [25]:
# Dropping duplicates after merging
merged_hotel_data = merged_hotel_data.drop_duplicates(subset='hotel_id')

In [26]:
# filter the merged data for Moscow
merged_hotel_data_moscow = merged_hotel_data.loc[(merged_hotel_data["city"].isin(['Moscow']))
                                                  ]
merged_hotel_data_moscow

,hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room,...,country,city_actual,rating_reviewcount,center1label,center2label,neighbourhood,ratingta,ratingta_count,distance_alter,accommodation_type
77569,10946,16,0,0% no offer,2018,3,1,0,1,1,...,Russia,Moscow,NaN,City centre,Solntsevo Exhibition Hall,Akademicheskiy,4.0,1.0,7.8,Hostel
77578,10947,59,1,15-50% offer,2017,12,0,1,1,0,...,Russia,Moscow,240.0,City centre,Solntsevo Exhibition Hall,Alexeevsky,3.0,2351.0,15.0,Hotel
77587,10948,40,1,15-50% offer,2018,4,1,0,1,0,...,Russia,Moscow,19.0,City centre,Solntsevo Exhibition Hall,Alexeevsky,4.0,34.0,15.0,Hotel
77596,10949,73,1,15-50% offer,2018,1,1,0,1,0,...,Russia,Moscow,31.0,City centre,Solntsevo Exhibition Hall,Alexeevsky,4.0,90.0,15.0,Hotel
77604,10950,40,1,15-50% offer,2017,12,0,1,1,0,...,Russia,Moscow,69.0,City centre,Solntsevo Exhibition Hall,Alexeevsky,3.5,230.0,15.0,Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80890,11341,466,0,0% no offer,2017,12,0,1,4,1,...,Russia,Moscow,9.0,City centre,Solntsevo Exhibition Hall,Yakimanka,3.5,242.0,10.0,Hotel
80894,11342,92,1,15-50% offer,2017,12,0,1,1,0,...,Russia,Moscow,14.0,City centre,Solntsevo Exhibition Hall,Yakimanka,NaN,NaN,10.0,Apartment
80903,11343,339,0,0% no offer,2017,12,0,1,4,0,...,Russia,Moscow,10.0,City centre,Solntsevo Exhibition Hall,Yakimanka,3.5,146.0,9.9,Hotel
80908,11344,311,1,15-50% offer,2017,12,0,1,4,0,...,Russia,Moscow,NaN,City centre,Solntsevo Exhibition Hall,Yakimanka,5.0,2.0,10.0,Inn


77569    NaN
77578    3.4
77587    3.7
77596    3.7
77604    4.0
        ... 
80890    4.6
80894    3.5
80903    3.5
80908    NaN
80917    NaN
Name: rating, Length: 400, dtype: float64